# SHREYA AKURATHI
# 200968188
# PHASE 2 - Write a python notebook to perform the recommendations and build an API for it. Demo of model, API and viva will be conducted in the lab on 13th May,2022.

As discussed in the previous notebook (Phase 1) , we will be going ahead with building the actual model (hybrid).
In this notebook I will start off by:
1. Splitting into train and test datasets
2. Analysing the cold start problem
3. Analysing for the best content based filtering approach
4. Analysisng for the best Collaborative Filtering approach
5. Combining the both 

## 1. Splitting into train and test datasets

In [1]:
import pandas as pd
import numpy as np

In [2]:
movies = pd.read_csv(r'C:\Users\shrey\OneDrive\Desktop\ML\ml-latest-small\movies.csv')
ratings = pd.read_csv(r'C:\Users\shrey\OneDrive\Desktop\ML\ml-latest-small\ratings.csv')
tags = pd.read_csv(r'C:\Users\shrey\OneDrive\Desktop\ML\ml-latest-small\tags.csv')
print('movies: ', movies.shape)
print('ratings: ', ratings.shape)
print('tags: ', tags.shape)

movies:  (9742, 3)
ratings:  (100836, 4)
tags:  (3683, 4)


In [3]:
df = pd.merge(ratings, movies, on='movieId' , how='left')

df.head(5)

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,3,4.0,964981247,Grumpier Old Men (1995),Comedy|Romance
2,1,6,4.0,964982224,Heat (1995),Action|Crime|Thriller
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),Mystery|Thriller
4,1,50,5.0,964982931,"Usual Suspects, The (1995)",Crime|Mystery|Thriller


In [4]:
df['genres'] = df['genres'].str.split('|')
df

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]"
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]"
2,1,6,4.0,964982224,Heat (1995),"[Action, Crime, Thriller]"
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]"
4,1,50,5.0,964982931,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]"
...,...,...,...,...,...,...
100831,610,166534,4.0,1493848402,Split (2017),"[Drama, Horror, Thriller]"
100832,610,168248,5.0,1493850091,John Wick: Chapter Two (2017),"[Action, Crime, Thriller]"
100833,610,168250,5.0,1494273047,Get Out (2017),[Horror]
100834,610,168252,5.0,1493846352,Logan (2017),"[Action, Sci-Fi]"


In [5]:
tags['tag'] = tags['tag'].str.split('|')
tags.drop('timestamp', axis=1, inplace=True)

In [6]:
tags = tags.groupby(['userId','movieId'])['tag'].apply(lambda x: ','.join(x.astype(str))).reset_index()
tags.head(5)

,userId,movieId,tag
0,2,60756,"['funny'],['Highly quotable'],['will ferrell']"
1,2,89774,"['Boxing story'],['MMA'],['Tom Hardy']"
2,2,106782,"['drugs'],['Leonardo DiCaprio'],['Martin Scors..."
3,7,48516,['way too long']
4,18,431,"['Al Pacino'],['gangster'],['mafia']"


In [7]:
df = pd.merge(df, tags, on=['userId','movieId'], how='left')

In [8]:
df.shape

(100836, 7)

In [9]:
df['tag'] = df['tag'].apply(lambda d: d if isinstance(d, list) else [])
df['genres'] = df['genres'].apply(lambda d: d if isinstance(d, list) else [])

In [10]:
df.head()

,userId,movieId,rating,timestamp,title,genres,tag
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",[]
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]",[]
2,1,6,4.0,964982224,Heat (1995),"[Action, Crime, Thriller]",[]
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]",[]
4,1,50,5.0,964982931,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]",[]


In [11]:
#split 
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(df, test_size=0.2, stratify=df.userId)


In [12]:
train_data = train_data.sort_values(['userId', 'movieId'])
train_data.head()

,userId,movieId,rating,timestamp,title,genres,tag
0,1,1,4.0,964982703,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]",[]
1,1,3,4.0,964981247,Grumpier Old Men (1995),"[Comedy, Romance]",[]
2,1,6,4.0,964982224,Heat (1995),"[Action, Crime, Thriller]",[]
3,1,47,5.0,964983815,Seven (a.k.a. Se7en) (1995),"[Mystery, Thriller]",[]
4,1,50,5.0,964982931,"Usual Suspects, The (1995)","[Crime, Mystery, Thriller]",[]


In [13]:
test_data = test_data.sort_values(['userId','movieId'])
test_data.head()

,userId,movieId,rating,timestamp,title,genres,tag
9,1,157,5.0,964984100,Canadian Bacon (1995),"[Comedy, War]",[]
17,1,316,3.0,964982310,Stargate (1994),"[Action, Adventure, Sci-Fi]",[]
19,1,349,4.0,964982563,Clear and Present Danger (1994),"[Action, Crime, Drama, Thriller]",[]
23,1,423,3.0,964982363,Blown Away (1994),"[Action, Thriller]",[]
32,1,590,4.0,964982546,Dances with Wolves (1990),"[Adventure, Drama, Western]",[]


In [14]:
train_data.to_csv('training_data1.csv', index = False)
test_data.to_csv('testing_data1.csv', index = False)

## 2.Analysing the cold start problem

In the blocks below I will try to analyse how different approaches solve the cold start problem.
The cold start problems occue when the recommender system is unable to form any relation between users and items, due to insuffiecient data/new users/low interaction.

I will compute the rsme and mean absolute error

In [15]:
import pickle
import os

import pandas as pd

from surprise import SVD, SVDpp
from surprise import KNNBasic, KNNBaseline, BaselineOnly
from surprise import Dataset                                                     
from surprise import Reader                                                      
from surprise import dump
from surprise.accuracy import rmse

In [16]:
def convert_traintest_dataframe_forsurprise(training_dataframe, testing_dataframe):
    reader = Reader(rating_scale=(0, 5))
    trainset = Dataset.load_from_df(training_dataframe[['userId', 'movieId', 'rating']], reader)
    testset = Dataset.load_from_df(testing_dataframe[['userId', 'movieId', 'rating']], reader)
    trainset = trainset.construct_trainset(trainset.raw_ratings)
    testset = testset.construct_testset(testset.raw_ratings)
    return trainset, testset

In [17]:
file_path_train = 'training_data.csv'
file_path_test = 'testing_data.csv'
traindf = pd.read_csv(file_path_train)
testdf = pd.read_csv(file_path_test)
trainset, testset = convert_traintest_dataframe_forsurprise(traindf, testdf)

In [ ]:
algo_svd = SVD()     
algo_svdpp = SVDpp()                                    
algo_knn = KNNBasic()


algo_svd.fit(trainset)                             
predictions_svd = algo_svd.test(testset)

algo_svdpp.fit(trainset)                             
predictions_svdpp = algo_svdpp.test(testset)

algo_knn.fit(trainset)
predictions_knn = algo_knn.test(testset)

#rmse(predictions_svd)
#rmse(predictions_knn)                                                                           

dump.dump('./dump_SVD', predictions_svd, algo_svd)
dump.dump('./dump_SVDpp', predictions_svdpp, algo_svdpp)
dump.dump('./dump_KNN', predictions_knn, algo_knn)

In [ ]:

df_svd = pd.DataFrame(predictions_svd, columns=['uid', 'iid', 'rui', 'est', 'details']) 
df_svdpp = pd.DataFrame(predictions_svdpp, columns=['uid', 'iid', 'rui', 'est', 'details'])        
df_knn = pd.DataFrame(predictions_knn, columns=['uid', 'iid', 'rui', 'est', 'detail'])

In [ ]:

sim_options = {'name': 'pearson_baseline',
               'user_based': False  # compute  similarities between items
               }
# algo = KNNBaseline(sim_options=sim_options)
algo_knnbaseline = KNNBaseline(sim_options=sim_options)
algo_knnbaseline.fit(trainset)
predictions_knnbaseline = algo_knnbaseline.test(testset)

In [ ]:
df_knnbaseline = pd.DataFrame(predictions_knnbaseline, columns=['uid', 'iid', 'rui', 'est', 'details']) 
df_knnbaseline['err'] = abs(df_knnbaseline.est - df_knnbaseline.rui)
df_knnbaseline['sqr_err'] = (df_knnbaseline.est - df_knnbaseline.rui)**2

In [ ]:
df_svd['err'] = abs(df_svd.est - df_svd.rui)
df_svdpp['err'] = abs(df_svdpp.est - df_svdpp.rui)
df_knn['err'] = abs(df_knn.est - df_knn.rui)

In [ ]:
df_svd['sqr_err'] = (df_svd.est - df_svd.rui)**2
df_svdpp['sqr_err'] = (df_svdpp.est - df_svdpp.rui)**2
df_knn['sqr_err'] = (df_knn.est - df_knn.rui)**2

In [ ]:
algo_baselineonly = BaselineOnly()
algo_baselineonly.fit(trainset)
predictions_baselineonly = algo_baselineonly.test(testset)

df_baselineonly = pd.DataFrame(predictions_baselineonly, columns=['uid', 'iid', 'rui', 'est', 'details']) 
df_baselineonly['err'] = abs(df_baselineonly.est - df_baselineonly.rui)
df_baselineonly['sqr_err'] = (df_baselineonly.est - df_baselineonly.rui)**2
df_baselineonly['iu'] = df_baselineonly.uid.apply(get_Iu)

In [ ]:
sim_options = {'name': 'pearson_baseline',
               'user_based': True  # compute  similarities between items
               }
algo_knnbaseline_user = KNNBaseline(sim_options=sim_options)
algo_knnbaseline_user.fit(trainset)
predictions_knnbaseline_user = algo_knnbaseline_user.test(testset)

df_knn_user = pd.DataFrame(predictions_knnbaseline_user, columns=['uid', 'iid', 'rui', 'est', 'details']) 
df_knn_user['err'] = abs(df_knn_user.est - df_knn_user.rui)
df_knn_user['sqr_err'] = (df_knn_user.est - df_knn_user.rui)**2
df_knn_user['iu'] = df_knn_user.uid.apply(get_Iu)

In [ ]:
df_svd.head()